In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pymorphy3
import json

In [20]:
# Загрузка необходимых ресурсов
nltk.download("punkt")
nltk.download("stopwords")

# Инициализация pymorphy3
morph = pymorphy3.MorphAnalyzer()

# Получение списка русских стоп-слов
stop_words = set(stopwords.words('russian'))

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [21]:
def lemmatize_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация
    words = word_tokenize(text, language="russian")
    
    # Удаление пунктуации и стоп-слов
    words = [word for word in words if word.isalnum() and word not in stop_words]
    
    # Лемматизация
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    
    return ' '.join(lemmatized_words)

In [22]:
# Функция для поиска по заголовку и получения определенных полей
def get_fields_by_title(json_data, title):
    for item in json_data['data']:
        if item['title'] == title:
            return {
                "question": f"{title}",
                "answer": item.get("description"),
                "url": item.get("url"),
            }
    return None

In [23]:
# Получаем путь к файлу JSON
json_file_path = './data-base/dataset.json'

# Открываем файл JSON и загружаем данные
with open(json_file_path, 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# Извлекаем все значения 'title' в массив
titles = [item['title'] for item in dataset['data']]

In [24]:
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [25]:
# Массив для лемматизированных вопросов
lemmatized_questions = []

# Цикл для лемматизации каждого вопроса
for question in titles:
    lemmatized_question = lemmatize_text(question)
    lemmatized_questions.append({"question": question, "lemmatized_question": lemmatized_question})

questions = [item["lemmatized_question"] for item in lemmatized_questions]


In [26]:
# Токенизация и векторизация
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

In [27]:
# Создание функции для нахождения ближайшего вопроса и всех вопросов на небольшом расстоянии от него
def find_closest_questions(user_question, extra_threshold=0.01):
    lemmatized_user_question = lemmatize_text(user_question)
    user_question_vec = vectorizer.transform([lemmatized_user_question])
    similarities = cosine_similarity(user_question_vec, X).flatten()
    
    # Находим индекс ближайшего вопроса
    closest_idx = np.argmax(similarities)
    max_similarity = similarities[closest_idx]
    
    # Устанавливаем пороговое значение
    threshold = max_similarity - extra_threshold
    
    # Находим все вопросы, которые находятся в пределах порогового значения
    closest_indices = np.where(similarities >= threshold)[0]
    return closest_indices

In [28]:
def get_answers(user_question, extra_threshold=0.01, max_answers=7):
    closest_indices = find_closest_questions(user_question, extra_threshold)
    return [lemmatized_questions[idx]["question"] for idx in closest_indices[:max_answers]]

In [29]:
# Пример использования
user_question = "какая налоговая ставка при оформлении ИП"
answers = get_answers(user_question, extra_threshold=0.1)
print(f"Вопрос: {user_question}\nОтветы:")

prompt = []

for answer in answers:
    result = get_fields_by_title(dataset, answer)
    prompt.append(result)

print(prompt)

Вопрос: какая налоговая ставка при оформлении ИП
Ответы:
[{'question': 'Какие документы нужны для оформления ИП?', 'answer': 'Для регистрации через сервис Тинькофф достаточно российского паспорта и СНИЛС. Как заполнить анкету для регистрации ИП Если регистрируетесь самостоятельно в налоговой, понадобятся следующие документы: заявление о регистрации в качестве ИП; копия российского паспорта; квитанция об уплате госпошлины, если регистрируетесь не через сайт, а лично или отправляете документы почтой.', 'url': 'https://www.tinkoff.ru/business/help/registration/registration/register-ip/opening-conditions/?card=q4'}]


In [30]:
json_file_path = './querry/prompt.json'

# Запись обновленных данных в JSON-файл
save_json(prompt, json_file_path)